In [123]:
# imports

import os
import pandas as pd
import re
import numpy as np

In [170]:
pd.set_option('display.max_rows', 32)

### Define functions to get files and manipulate dataframes

In [262]:
# get all files full paths in data folder
def get_files(data_dir='data'):
    is_dir = os.path.isdir(data_dir)
    if not is_dir:
        print(f'\"{os.getcwd()}/{data_dir}\" directory does not exist, please specify correct data directory.')
        exit()
    files = []
    for root, dirs, filenames in os.walk(data_dir):
        for name in filenames:
            f = os.path.join(root, name)
            if f.endswith('data.csv'):
                files.append(f)
            else:
                continue

    return files


# rename columns: make lowercase, replace 'space' with '_'
def rename_cols(df):
    cols = []
    for col in df.columns.to_list():
        col = col.strip().replace(' ', '_').lower()
        cols.append(col)
    df.columns = cols


# populate list by list of files grouped by property type
def files_list():
    files = sorted(get_filenames())
    df_list = []
    i_temp = 0
    tmp_list = []
    for i in range(len(files)-1):
        current_name = files[i_temp].split('/')[-1].split('_')[0]
        next_name = files[i+1].split('/')[-1].split('_')[0]
        if current_name == 'Rooms for rent':
            df_list.append(files[i_temp:])
            break
        elif current_name == next_name:
            tmp_list.append(files[i+1])
        else:
            tmp_list.insert(0, files[i_temp])
            df_list.append(tmp_list)
            tmp_list = []
            i_temp = i+1
    return df_list


# concatenate csv files that are same property types
def df_concat(files_list):
    df_merged = pd.DataFrame()
    for file in files_list:
        region = file.split('/')[-2]
        df_to_merge = pd.read_csv(file)
        if len(df_to_merge) < 1:
            continue
        df_to_merge['region'] = region
        frames = [df_merged, df_to_merge]
        df_merged = pd.concat(frames, ignore_index=True)
    return df_merged


# fix data displacement occured while scraping
def fix_displacement(df):
    df_fix = df[~df['Links'].str.startswith('https', na=False)].loc[:, 'Datetime':]
    links = []
    date = []

    for row in df_fix.astype(str).values:
        l = len(row)
        for i in range(len(row)):
            if row[i].startswith('https'):
                links.append(row[i])
                if len(links) > len(date):
                    date.append(tmp_date)
                break
            if re.match(r'^[A-Z][a-z]{2}-\d+-\d+_\d+-\d+', row[i]):
                tmp_date = row[i]
                date.append(row[i])

    df.loc[~df['Links'].str.startswith('https', na=False), 'Datetime'] = date
    df.loc[~df['Links'].str.startswith('https', na=False), 'Links'] = links
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    return df


def clean_currency(x):
    """ If the value is a string, then remove currency symbol and delimiters
    otherwise, the value is numeric and can be converted
    """
    # df_merged['Price'] = df_merged['Price'].replace({'\$': '', '֏': '', '₽': '', '€': '', ',': ''}, regex=True)
    if isinstance(x, str):
        return x.replace('$', '').replace('֏', '').replace('₽', '').replace('€', '').replace(',', '')

    
# split 'Price' column (e.g. 10000 monthly) to 2 columns and reorder df columns
def split_price_col(df):
    df[['Price', 'Duration']] = df.Price.str.split(expand=True)
    df['Price'] = df.loc[:, 'Price'].astype(int)
    cols = df.columns.to_list()
    idx_currency = cols.index('Currency')
    idx_duration = cols.index('Duration')
    cols.insert(idx_currency+1, 'Duration')
    cols.pop()
    df = df[cols]
    return df


def missing_values(df):    
    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_value_df = pd.DataFrame({
                                     'percent_missing': percent_missing}, index=None)
    return missing_value_df


def multi_trans(df):
    df1['New Construction'] = pd.Series(np.where(df1['New Construction'].values == 'Yes', 1, 0),
          df1.index, dtype=int)
    df1['Elevator'] = pd.Series(np.where(df1['Elevator'].values == 'Available', 1, 0),
          df1.index, dtype=int)
    
    df1['Floors in the Building'] = df1['Floors in the Building'].replace({'\+': ''}, regex=True)
    df1['Floors in the Building'] = df1['Floors in the Building'].fillna(-1)
    df1['Floors in the Building'] = df1['Floors in the Building'].astype(int)
    df1['Floors in the Building'] = df1['Floors in the Building'].replace('-1', np.nan)
    df1['Floors in the Building']

def drop_cols(df):
    df = df.dropna(thresh=int(df.shape[0]*0.2), axis=1)
    return df


In [265]:
files = files_list()
df1 = df_concat(files[0])
df1 = fix_displacement(df1)
df1['Price'] = df1['Price'].apply(clean_currency)
df1 = split_price_col(df1)
rename_cols(df1)

/tmp/ipykernel_97359/2563175892.py:55: DtypeWarning: Columns (3,35,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df_to_merge = pd.read_csv(file)


In [271]:
df1 = drop_cols(df1)

In [293]:
df1['floor_area'].astype('str').str.extractall('(\d+)').astype(int).isna()==False

0    0
dtype: int64

In [192]:
df1['amenities']

0                                               Television
1                                                      NaN
2        Television, air conditioner, internet, parking...
3                                                      NaN
4                                                      NaN
                               ...                        
33132                                                  NaN
33133                                                  NaN
33134                                                  NaN
33135                                                  NaN
33136    Television, air conditioner, internet, parking...
Name: amenities, Length: 33137, dtype: object

In [92]:
# check if changes applied
df1[~df1['Links'].str.startswith('https', na=False)].loc[:, 'Datetime':]
df1[['Price', 'Currency']]

,Price,Currency
0,50000,AMD
1,60000,AMD
2,16000,AMD
3,250000,AMD
4,250000,AMD
...,...,...
33132,1900,USD
33133,200000,AMD
33134,240000,AMD
33135,12000,AMD


In [85]:
df2 = df_concat(files[1])
df2 = fix_displacement(df2)
df2['Price'] = df2['Price'].apply(clean_currency)

In [87]:
df2[['Price', 'Currency']]

,Price,Currency
0,16000,USD
1,100000,USD
2,115000,USD
3,600,USD
4,15500000,AMD
...,...,...
31359,43000,USD
31360,28500000,AMD
31361,62000,USD
31362,65000,USD
